In [1]:
# Define SCHOOL, data_path, and output_path
SCHOOL = "McGill"
data_path = f"bias_processing/data/1/{SCHOOL.lower()}_dataset.csv"
output_path = f"bias_processing/data/3/{SCHOOL.lower()}_dataset_summarizer.csv"

In [ ]:
"""
Load in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns
4 topics ("Israel", "Palestine", "India", "China")
3 hypotheses for sentiment (Positive, Negative, Neutral)
3 levels of granularity
Compute sentiment for entire article
Summarizes each paragraph using an ML summarizing model, and join those summaries to one body of text. Compute sentiment for this new article version.
Summarize the entire article in one go using the same ML model. Compute sentiment for this new article version
Save a new csv with these added columns
"""

In [2]:
%pip install nltk pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\stacy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import os
import nltk
os.environ['NLTK_DATA'] = r"\Users\stacy\Desktop\Olami Project\Olami"
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\stacy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stacy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from statistics import mean

# Instantiate the sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to return the sentiment of a text
def get_sentiment(text, agg_func):
    scores = [sid.polarity_scores(sentence) for sentence in nltk.sent_tokenize(text)]
    
    if agg_func == 'average':
        return mean([score['compound'] for score in scores])
    elif agg_func == 'max':
        return max([score['compound'] for score in scores])
    else:
        raise ValueError("Invalid agg_func. Expected 'average' or 'max'")

# Load the csv
df = pd.read_csv(data_path)

# Process sentiment analysis
for keyword in ["Israel", "Palestine", "India", "China"]:
    for agg_func in ['average', 'max']:
        # Filter articles by keyword
        keyword_df = df[df['keyword'] == keyword]
        
        # Compute sentiment for each level of granularity
        df[f'{keyword}_article_{agg_func}'] = keyword_df['article'].apply(lambda text: get_sentiment(text, agg_func))
        df[f'{keyword}_paragraph_{agg_func}'] = keyword_df['article'].apply(lambda text: max([get_sentiment(paragraph, agg_func) for paragraph in text.split('\n\n') if paragraph]))
        df[f'{keyword}_sentence_{agg_func}'] = keyword_df['article'].apply(lambda text: max([get_sentiment(sentence, agg_func) for sentence in nltk.sent_tokenize(text)]))

# Save the output DataFrame into a new CSV file
df.to_csv(output_path, index=False)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\stacy/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\stacy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
